## 路径还原-精确匹配

注：存在 86 个重复三元组，提交不知道会不会遇到这些情形
- 比如假设 triple1 即为 en 也为 zh，那么
- en -> zh -> en 和 zh -> zh -> en 都将作为答案

In [6]:
include("../src/CCKS-mKGQA.jl")
include("../src/tools/regexpr.jl") # 正则表达
include("../src/loaddata/extractdata.jl") # 原始数据
include("../src/loaddata/translatedata.jl") # 翻译数据
include("../src/tools/ettalign.jl") # 对齐工具
include("../src/tools/distance.jl") # 距离函数
include("../src/process/submitpath.jl") # 数据提交
include("../src/datatype.jl")
mkpath("submit")

"submit"

In [7]:
# illpath = "ILLs_1"
# illpath = "ILLs_2"
# illpath = "ILLs_3"
# illpath = "ILLs_4"
# illpath = "ILLs_official"
illpath = "ILLs_wiki"
include("../src/loaddata/illsdata.jl");

In [8]:
# 元组逆向
println("重复元组的信息")
sort_count(length.(values(illsdata.triple_ills2raw)))

重复元组的信息


OrderedDict{Int64, Int64} with 4 entries:
  1 => 255662
  2 => 5857
  3 => 31
  4 => 4

### 精确匹配

In [9]:
submit_id = "0728"
sols = split.(strip.(split(strip(read(open("predict/predict_data_bertsim.txt", "r"), String)), '\n')),'\t')
noans = "<http://dbpedia.org/resource/Francis_Russell,_Marquess_of_Tavistock>#<http://dbpedia.org/property/parents>#<http://dbpedia.org/resource/John_Russell,_4th_Duke_of_Bedford>#<http://zh.dbpedia.org/resource/第四代贝德福德公爵约翰·罗素>#<http://zh.dbpedia.org/property/successor>#<http://zh.dbpedia.org/resource/丹尼尔·芬奇，第八代温奇尔西伯爵>";

# 返回结果
res = Vector{String}(undef, 1500)
fails, notgood = Int[], Int[]
## 精确匹配
for (i, sol) in enumerate(sols)
    que, rels = ill_valid_ques[i], sol[3:end]
    ner, rawner = ill_valid_ners[i], valid_ners[i]
    # 精确路径
    submits = precise_submit(ner, rels, illsdata)
    isempty(submits) && (push!(fails, i);continue) # 无解
    bettersubmits = filter(contains(rawner), submits) # 不需要实体链接
    isempty(bettersubmits) ? push!(notgood, i) : (submits = bettersubmits)
    res[i] = first(submits)
end
println(length(fails), '\t', length(notgood))

## 剩下部分放弃
res[fails] .= noans
submit_io = open("submit/submit_$submit_id.txt", "w")
demo_io = open("submit_test/demo_$submit_id.txt", "w")
println(submit_io, "id\tans_path")
for (i, sol) in enumerate(res)
    println(submit_io, i-1, '\t', sol)
    info = triple_info.(partition(split(sol, '#'), 3))
    sol == noans && continue
    println(demo_io, i-1, '\t', join(mt_valid_ques_ner[i], '\t'))
    println.(Ref(demo_io), info)
    println(demo_io)
end
close(submit_io)
close(demo_io)

207	10


In [8]:
# 准确率
0.6573 * 1500 / (1500 - 286)

0.8121499176276772

### 数据分析

In [10]:
fail_io = open("fails.txt", "w")
for (que, ner, rels...) in sols[fails]
    println(fail_io, que, '\t', ner)
    println(fail_io, join(rels, '\t'))
    println(fail_io)
end
# println.(Ref(fail_io), join.(sols[fails], '\t'))
close(fail_io)

In [55]:
# 数据比对
cmp = open("submit_test/compare_yes_tod.txt", "w")
i = j = 0
name1, name2 = "YESTODAY", "TODAY"
for (k, que, ner, res1, res2) in zip(1:1500, mt_valid_ques, valid_ners, yestoday, today)
    res1 == res2 && continue
    println(cmp, k, '\t', que, '\t', ner)
    if res1 == noans
        println(cmp, "$name1\nNONE\n$name2")
        println.(Ref(cmp), triple_info.(partition(split(res2, '#'), 3)))
        j += 1
    elseif res2 == noans
        println(cmp, "$name1")
        println.(Ref(cmp), triple_info.(partition(split(res1, '#'), 3)))
        println(cmp, "$name2\nNONE")
    else
        println(cmp, "$name1")
        println.(Ref(cmp), triple_info.(partition(split(res1, '#'), 3)))
        println(cmp, "$name2")
        println.(Ref(cmp), triple_info.(partition(split(res2, '#'), 3)))
    end
    println(cmp)
    i += 1
end
close(cmp)
println("数据差异 $i, $name2 新解数 $j")

数据差异 153, TODAY 新解数 24
